In [1]:
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
print(tf.__version__)

1.0.0-rc0


In [3]:
# Try using the basic tensorflow operation to perform linear regression on Californic Housing Data

housing = fetch_california_housing()

In [4]:
m, n = housing.data.shape
scaler = StandardScaler()
scaler.fit(housing.data)
scaled_housing_data = scaler.transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [23]:
# Convert numpy matrices into tensors
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target, shape=(m,1), dtype=tf.float32, name='y')

theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0,seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m*tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MES=", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MES= 2.75443
Epoch 100 MES= 0.632222
Epoch 200 MES= 0.57278
Epoch 300 MES= 0.558501
Epoch 400 MES= 0.549069
Epoch 500 MES= 0.542288
Epoch 600 MES= 0.537379
Epoch 700 MES= 0.533822
Epoch 800 MES= 0.531243
Epoch 900 MES= 0.529371


In [27]:
# Next, let's replace the optimizer with embeded autodifferentiation
tf.reset_default_graph()

# Convert numpy matrices into tensors
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target, shape=(m,1), dtype=tf.float32, name='y')

theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init= tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("# of epoch", epoch, "mse", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

# of epoch 0 mse 2.75443
# of epoch 100 mse 0.632222
# of epoch 200 mse 0.57278
# of epoch 300 mse 0.558501
# of epoch 400 mse 0.549069
# of epoch 500 mse 0.542288
# of epoch 600 mse 0.537379
# of epoch 700 mse 0.533822
# of epoch 800 mse 0.531243
# of epoch 900 mse 0.529371


In [77]:
# Try mini-batch gradient descent
# Instead of feeding in X, y as constant, we need to feed them in as placeholder with specific shape
tf.reset_default_graph()
n_epochs = 10
learning_rate = 0.001

batch_size = 100
n_batches = int(np.ceil(m/batch_size))

X = tf.placeholder(tf.float32, shape=(None,n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None,1), name='y')

theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

def batch_fetch(epoch, batch_index, batch_size):
    # Input epoch number, batch_idex
    # determine the random shuffling by epoch number
    np.random.seed(epoch * n_batches + batch_index)          # not shown in the book
    indices = np.random.randint(m, size=batch_size)          # not shown
    X_batch = scaled_housing_data_plus_bias[indices]   # not shown
    y_batch = housing.target.reshape(-1, 1)[indices]   # not shown
    return X_batch, y_batch

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = batch_fetch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch,y:y_batch})
    
    best_theta = theta.eval()

In [78]:
best_theta

array([[ 2.06703067],
       [ 0.82357562],
       [ 0.11846008],
       [-0.23221512],
       [ 0.30244407],
       [ 0.00438231],
       [-0.03862762],
       [-0.91111773],
       [-0.88494778]], dtype=float32)

## Visualizing the Graph and Training Curves Using TensorBoard

In [99]:
# Creat specific log directory for storing the training stats
# A good way to avoid overlapping is to name with time information

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [100]:
# We need to add function to evaluate the summary file
# We also need to convert summary file into log file with File writer
# Let's rewrite the training with Adam for california housing data
m, n = housing.data.shape
tf.reset_default_graph()

n_epochs = 10
learning_rate = 0.001
batch_size = 100
batch_num = int(np.ceil(m/batch_size))

X = tf.placeholder(dtype=tf.float32, shape=[None, n+1], name='X')
y = tf.placeholder(dtype=tf.float32, shape=[None, 1], name='y')

theta = tf.Variable(tf.random_uniform([n+1, 1], -0.1, 0.1, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_sum(tf.square(error), name='mse')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(batch_num):
            np.random.seed(epoch * batch_size + batch_index)
            rand_idx = np.random.randint(m, size=batch_size)
            X_batch = scaled_housing_data_plus_bias[rand_idx]
            y_batch = housing.target.reshape(-1, 1)[rand_idx]
            
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
                step = epoch * batch_num + batch_index
                file_writer.add_summary(summary_str, step)
                
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    
    best_theta = theta.eval()
    
file_writer.close()

NameError: name 'source' is not defined